In [ ]:
import torch
from torch import nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset
import numpy as np
import math
import os
import random
import tqdm
import gzip
import time

!pip install einops
from einops import rearrange, repeat, pack, unpack, einsum
from einops.layers.torch import Rearrange


from functools import partial, wraps
from contextlib import contextmanager, ExitStack
from pathlib import Path
from filelock import FileLock
import pickle

import transformers
from transformers import AutoTokenizer

!pip install faiss-gpu
import faiss

!pip install datasets
import datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.6/44.6 kB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.5/85.5 MB 18.7 MB/s eta 0:00:00:00:0100:01


In [ ]:
"""
This line of code checks whether a CUDA-enabled GPU is available on the system.
If a GPU is available, it sets the device to 'cuda'; otherwise, it sets it to 'cpu'.
"""
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
class RelativePosition(nn.Module):
    """
    RelativePosition module computes relative positions between sequence elements and generates corresponding relative position embeddings.

    Args:
        rp_scale (float): Scaling factor for the relative position embeddings.
        num_buckets (int): Number of buckets for discretizing relative positions.
        rp_max_distance (int): Maximum relative distance considered.
        heads (int): Number of attention heads.

    Attributes:
        scale (float): Scaling factor for the relative position embeddings.
        num_buckets (int): Number of buckets for discretizing relative positions.
        rp_max_distance (int): Maximum relative distance considered.
        relative_attention_embedding (torch.nn.Embedding): Embedding layer for storing relative position embeddings.

    Methods:
        relative_position_bucket: Computes bucket indices for relative positions.
        forward: Forward pass of the module.
    """

    def __init__(
        self,
        rp_scale,
        num_buckets=32,
        rp_max_distance=128,
        heads=8
    ):
        """
        Initializes the RelativePosition module.

        Args:
            rp_scale (float): Scaling factor for the relative position embeddings.
            num_buckets (int): Number of buckets for discretizing relative positions.
            rp_max_distance (int): Maximum relative distance considered.
            heads (int): Number of attention heads.
        """
        super().__init__()
        self.scale = rp_scale
        self.num_buckets = num_buckets
        self.rp_max_distance = rp_max_distance
        self.relative_attention_embedding = nn.Embedding(num_buckets, heads)

    def relative_position_bucket(self, relative_position_matrix):
        """
        Computes bucket indices for given relative position matrix.

        Args:
            relative_position_matrix (torch.Tensor): Matrix of relative positions.

        Returns:
            torch.Tensor: Bucket indices for the given relative positions.
        """
        n = -relative_position_matrix
        n = torch.max(n, torch.zeros_like(n))

        max_exact = self.num_buckets // 2

        is_small = n < max_exact
        val_if_large = max_exact + (torch.log(n.float() / max_exact) / math.log(self.rp_max_distance / max_exact) * (self.num_buckets - max_exact)).long()
        val_if_large = torch.min(val_if_large, torch.full_like(val_if_large, self.num_buckets - 1))

        return torch.where(is_small, n, val_if_large)

    def forward(self, sequence_length, device):
        """
        Forward pass of the RelativePosition module.

        Args:
            sequence_length (int): Length of the sequence.
            device (torch.device): Device to perform computations.

        Returns:
            torch.Tensor: Relative position embeddings scaled by the scaling factor.
        """
        sequence_pos = torch.arange(sequence_length, dtype=torch.long, device=device)
        context_pos = torch.arange(2 * sequence_length, dtype=torch.long, device=device)
        sequence_rel_pos = rearrange(sequence_pos, 'i -> i 1')
        context_rel_pos = rearrange(context_pos, 'j -> 1 j')
        rel_pos = context_rel_pos - sequence_rel_pos

        position_bucket_indices = self.relative_position_bucket(rel_pos)

        rp_values = self.relative_attention_embedding(position_bucket_indices)
        rp_values = rearrange(rp_values, 'i j h -> () h i j')
        return rp_values * self.scale

In [ ]:
class KNN():
    """
    KNN (K-Nearest Neighbors) class for efficient nearest neighbor search.

    Args:
        dim (int): Dimensionality of the data vectors.
        max_memories (int): Maximum number of data vectors to be stored in memory.

    Attributes:
        dim (int): Dimensionality of the data vectors.
        max_memories (int): Maximum number of data vectors to be stored in memory.
        shape (tuple): Shape of the memory map storing the data vectors.
        db_offset (int): Offset for indexing the memory map.
        db_filepath (str): Filepath for the memory map.
        db (np.memmap): Memory map for storing data vectors.
        index (faiss.IndexFlatL2): FAISS index for fast nearest neighbor search.

    Methods:
        add_to_db: Add new data vectors to the memory map.
        search_and_retrieve: Perform nearest neighbor search on query vectors.
        add: Add new data vectors to the memory map and update the index.
        search: Perform nearest neighbor search on query vectors.
        clear: Clear the memory map and reset the index.
    """

    def __init__(
        self,
        dim,
        max_memories,
        ):
        """
        Initializes the KNN object.

        Args:
            dim (int): Dimensionality of the data vectors.
            max_memories (int): Maximum number of data vectors to be stored in memory.
        """
        self.dim = dim
        self.max_memories = max_memories
        self.shape = (max_memories, 2, dim)
        self.db_offset = 0
        self.db_filepath = "./memory.memmap"
        self.db = np.memmap(self.db_filepath, mode='w+', dtype=np.float32, shape=self.shape)
        self.index = faiss.IndexFlatL2(dim)

    def add_to_db(self, new_data):
        """
        Add new data vectors to the memory map.

        Args:
            new_data (torch.Tensor): New data vectors to be added.
        """
        new_data_len = new_data.shape[0]
        ids = (np.arange(new_data_len) + self.db_offset)
        self.db[ids] = new_data.detach().cpu().numpy()
        self.db_offset += new_data_len
        # Write to file
        self.db.flush()

    def search_and_retrieve(self, query_vecs, topk):
        """
        Perform nearest neighbor search on query vectors.

        Args:
            query_vecs (np.ndarray): Query vectors for nearest neighbor search.
            topk (int): Number of nearest neighbors to retrieve.

        Returns:
            np.ndarray: Nearest neighbor key-value pairs.
        """
        query_vecs = query_vecs
        distances, indices = self.index.search(query_vecs, topk)
        kvs = self.db[indices]
        return kvs

    def add(self, new_data):
        """
        Add new data vectors to the memory map and update the index.

        Args:
            new_data (torch.Tensor): New data vectors to be added.
        """
        # Input is b n 2 d, flatten to (b n) 2 d
        new_data = new_data.flatten(0, 1)
        # Add to db
        self.add_to_db(new_data)
        # Only keys are used in knn index
        keys, vals = new_data.unbind(dim=-2)
        keys = keys.detach().cpu().numpy()
        # Add (b n) d tensors to index
        keys = np.ascontiguousarray(keys)
        # Add to index
        self.index.add(keys)

    def search(self, query_vecs, topk):
        """
        Perform nearest neighbor search on query vectors.

        Args:
            query_vecs (torch.Tensor): Query vectors for nearest neighbor search.
            topk (int): Number of nearest neighbors to retrieve.

        Returns:
            torch.Tensor: Nearest neighbor key-value pairs.
        """
        query_batch_size, query_seq_len = query_vecs.shape[0], query_vecs.shape[1]
        device = query_vecs.device
        # Input is b n d, flatten to (b n) d
        query_vecs = query_vecs.flatten(0, 1)
        kvs = self.search_and_retrieve(np.ascontiguousarray(query_vecs.detach().cpu().numpy()), topk)
        # kvs are (b n) k 2 d, unflatten to b n k 2 d
        kvs = torch.tensor(kvs)
        kvs = torch.unflatten(kvs, 0, (query_batch_size, query_seq_len))
        return kvs.to(device)

    def clear(self):
        """
        Clear the memory map and reset the index.
        """
        self.index.reset()
        self.db[:] = 0
        self.db_offset = 0


In [ ]:


class XLAttention(nn.Module):
    """
    XLAttention module implements the attention mechanism used in XLNet.

    Args:
        embedding_dimension (int): Dimensionality of the input embeddings.
        heads (int): Number of attention heads.
        head_dimension (int): Dimensionality of each attention head.
        dropout (float): Dropout probability.

    Attributes:
        heads (int): Number of attention heads.
        dropout (nn.Dropout): Dropout layer.
        scale (float): Scaling factor for attention scores.
        query_matrix (nn.Linear): Linear layer for query projection.
        key_matrix (nn.Linear): Linear layer for key projection.
        value_matrix (nn.Linear): Linear layer for value projection.
        output_matrix (nn.Linear): Linear layer for output projection.

    Methods:
        forward: Forward pass of the XLAttention module.
    """

    def __init__(
        self,
        embedding_dimension,
        heads=8,
        head_dimension=64,
        dropout=0.,
    ):
        """
        Initializes the XLAttention module.

        Args:
            embedding_dimension (int): Dimensionality of the input embeddings.
            heads (int): Number of attention heads.
            head_dimension (int): Dimensionality of each attention head.
            dropout (float): Dropout probability.
        """
        super().__init__()
        self.heads = heads
        self.dropout = nn.Dropout(dropout)
        self.scale = head_dimension ** -0.5

        self.query_matrix = nn.Linear(embedding_dimension, self.heads * head_dimension)
        self.key_matrix = nn.Linear(embedding_dimension, self.heads * head_dimension)
        self.value_matrix = nn.Linear(embedding_dimension, self.heads * head_dimension)
        self.output_matrix = nn.Linear(self.heads * head_dimension, embedding_dimension)

    def forward(
        self,
        x,  # batch_size, sequence_length, embedding_dimension
        relative_positions=None,
        xl_memory=None
    ):
        """
        Forward pass of the XLAttention module.

        Args:
            x (torch.Tensor): Input tensor of shape (batch_size, sequence_length, embedding_dimension).
            relative_positions (torch.Tensor): Tensor containing relative positions.
            xl_memory (torch.Tensor): XL memory for cross-attention.

        Returns:
            torch.Tensor: Output tensor.
            torch.Tensor: XL memory to add.
        """
        device = x.device
        queries = self.query_matrix(x)
        keys = self.key_matrix(x)
        values = self.value_matrix(x)

        queries = queries * self.scale

        if xl_memory is not None:
            k_xl, v_xl = xl_memory.unbind(dim=-2)
            keys = torch.cat((k_xl, keys), dim=-2)
            values = torch.cat((v_xl, values), dim=-2)
            xl_sequence_length = k_xl.shape[1]

        queries = rearrange(queries, 'b t (h d) -> b h t d', h=self.heads)
        keys = rearrange(keys, 'b t (h d) -> b h t d', h=self.heads)
        qk = einsum(queries, keys, 'b h i d, b h j d -> b h i j')

        i, j = qk.shape[-2:]
        if relative_positions is not None:
            qk = relative_positions[..., -i:, -j:] + qk

        qk = qk * self.scale

        mask = torch.ones((i, j), dtype=torch.bool, device=device).triu(j - i + 1)
        qk = qk.masked_fill(mask, float('-inf'))

        qk = F.softmax(qk, dim=-1)
        qk = self.dropout(qk)

        values = rearrange(values, 'b t (h d) -> b h t d', h=self.heads)
        qkv = qk @ values
        qkv = rearrange(qkv, 'b h t d -> b t (h d)')

        out = self.output_matrix(qkv)

        keys = rearrange(keys, 'b h t d -> b t (h d)', h=self.heads)
        values = rearrange(values, 'b h t d -> b t (h d)', h=self.heads)
        kv_memories = torch.stack((keys, values), dim=-2)

        if xl_memory is not None:
            xl_memories, current_input = kv_memories[:, :-xl_sequence_length], kv_memories[:, -xl_sequence_length:]
            kv_to_add_xl = current_input
        else:
            kv_to_add_xl = kv_memories

        return out, kv_to_add_xl


In [ ]:
class KNNAttention(nn.Module):
    """
    KNNAttention module implements the attention mechanism with K-Nearest Neighbors (KNN) retrieval.

    Args:
        embedding_dimension (int): Dimensionality of the input embeddings.
        knn (KNN): KNN object for memory retrieval.
        heads (int): Number of attention heads.
        head_dimension (int): Dimensionality of each attention head.
        topk_retrieved_memories (int): Number of top memories to retrieve.
        dropout (float): Dropout probability.

    Attributes:
        heads (int): Number of attention heads.
        scale (float): Scaling factor for attention scores.
        dropout (nn.Dropout): Dropout layer.
        query_matrix (nn.Linear): Linear layer for query projection.
        key_matrix (nn.Linear): Linear layer for key projection.
        value_matrix (nn.Linear): Linear layer for value projection.
        output_matrix (nn.Linear): Linear layer for output projection.
        gate_bias (nn.Parameter): Learnable bias for gating mechanism.
        topk_retrieved_memories (int): Number of top memories to retrieve.
        knn (KNN): KNN object for memory retrieval.

    Methods:
        forward: Forward pass of the KNNAttention module.
    """

    def __init__(
        self,
        embedding_dimension,
        knn,
        heads=8,
        head_dimension=64,
        topk_retrieved_memories=3,
        dropout=0.
    ):
        """
        Initializes the KNNAttention module.

        Args:
            embedding_dimension (int): Dimensionality of the input embeddings.
            knn (KNN): KNN object for memory retrieval.
            heads (int): Number of attention heads.
            head_dimension (int): Dimensionality of each attention head.
            topk_retrieved_memories (int): Number of top memories to retrieve.
            dropout (float): Dropout probability.
        """
        super().__init__()
        self.heads = heads
        self.scale = head_dimension ** -0.5
        self.dropout = nn.Dropout(dropout)

        self.query_matrix = nn.Linear(embedding_dimension, heads * head_dimension)
        self.key_matrix = nn.Linear(embedding_dimension, heads * head_dimension)
        self.value_matrix = nn.Linear(embedding_dimension, heads * head_dimension)
        self.output_matrix = nn.Linear(heads * head_dimension, embedding_dimension)

        self.gate_bias = nn.Parameter(torch.randn(self.heads, 1, 1))
        self.topk_retrieved_memories = topk_retrieved_memories
        self.knn = knn

    def forward(
        self,
        x,  # batch_size, sequence_length, embedding_dimension
        relative_positions=None,
        xl_memory=None
    ):
        """
        Forward pass of the KNNAttention module.

        Args:
            x (torch.Tensor): Input tensor of shape (batch_size, sequence_length, embedding_dimension).
            relative_positions (torch.Tensor): Tensor containing relative positions.
            xl_memory (torch.Tensor): XL memory for cross-attention.

        Returns:
            torch.Tensor: Output tensor.
            torch.Tensor: XL memory to add.
        """
        device = x.device
        batch_size, sequence_length = x.shape[:2]
        queries = self.query_matrix(x)
        keys = self.key_matrix(x)
        values = self.value_matrix(x)

        queries = F.normalize(queries, dim=-1)
        keys = F.normalize(keys, dim=-1)

        if xl_memory is not None:
            k_xl, v_xl = xl_memory.unbind(dim=-2)
            keys = torch.cat((k_xl, keys), dim=-2)
            values = torch.cat((v_xl, values), dim=-2)
            xl_sequence_length = k_xl.shape[1]

        queries = rearrange(queries, 'b t (h d) -> b h t d', h=self.heads)
        keys = rearrange(keys, 'b t (h d) -> b h t d', h=self.heads)
        qk = einsum(queries, keys, 'b h i d, b h j d -> b h i j')

        i, j = qk.shape[-2:]
        if relative_positions is not None:
            qk = relative_positions[..., -i:, -j:] + qk

        qk = qk * self.scale

        mask = torch.ones((i, j), dtype=torch.bool, device=device).triu(j - i + 1)
        qk = qk.masked_fill(mask, float('-inf'))

        qk = F.softmax(qk, dim=-1)

        qk = self.dropout(qk)

        values = rearrange(values, 'b t (h d) -> b h t d', h=self.heads)
        qkv = qk @ values

        if self.knn.index.ntotal > 0:
            queries = rearrange(queries, 'b h t d -> b t (h d)')
            mem_kv = self.knn.search(queries, topk=self.topk_retrieved_memories)
            mem_k, mem_v = mem_kv.unbind(dim=-2)
            mem_k = rearrange(mem_k, 'b t k (h d) -> b h t k d', h=self.heads)
            mem_v = rearrange(mem_v, 'b t k (h d) -> b h t k d', h=self.heads)

            queries = rearrange(queries, 'b t (h d) -> b h t d', h=self.heads)
            mem_qk = einsum(queries, mem_k, 'b h t d, b h t k d -> b h t k')
            mem_qk = mem_qk * self.scale

            mem_qk = F.softmax(mem_qk, dim=-1)
            mem_qk = self.dropout(mem_qk)
            mem_qkv = einsum(mem_qk, mem_v, 'b h t k, b h t k d -> b h t d')

            combined_qkv = mem_qkv * self.gate_bias + qkv * (1 - self.gate_bias)
            combined_qkv = rearrange(combined_qkv, 'b h t d -> b t (h d)')
            out = self.output_matrix(combined_qkv)
        else:
            qkv = rearrange(qkv, 'b h t d -> b t (h d)')
            out = self.output_matrix(qkv)

        keys = rearrange(keys, 'b h t d -> b t (h d)', h=self.heads)
        values = rearrange(values, 'b h t d -> b t (h d)', h=self.heads)
        kv_memories = torch.stack((keys, values), dim=-2)

        if xl_memory is not None:
            xl_memories, current_kv = kv_memories[:, :-xl_sequence_length], kv_memories[:, -xl_sequence_length:]
        else:
            current_kv = kv_memories

        self.knn.add(current_kv)

        return out, current_kv

In [ ]:
class Block(nn.Module):
    """
    Block module defines a single block in the transformer architecture.

    Args:
        embedding_dimension (int): Dimensionality of the input embeddings.
        attention_type (nn.Module): Type of attention mechanism to be used.
        dropout (float): Dropout probability.

    Attributes:
        attention (nn.Module): Attention mechanism.
        dim (int): Dimensionality of the input embeddings.
        norm (nn.LayerNorm): Layer normalization.
        ff_block (nn.Sequential): Feed-forward block.

    Methods:
        forward: Forward pass of the Block module.
    """

    def __init__(self, embedding_dimension, attention_type, dropout=0.):
        """
        Initializes the Block module.

        Args:
            embedding_dimension (int): Dimensionality of the input embeddings.
            attention_type (nn.Module): Type of attention mechanism to be used.
            dropout (float): Dropout probability.
        """
        super().__init__()
        self.attention = attention_type
        self.dim = embedding_dimension
        self.norm = nn.LayerNorm(self.dim)

        self.ff_block = nn.Sequential(
            nn.LayerNorm(self.dim),
            nn.Linear(self.dim, self.dim * 4),
            nn.GELU(),
            nn.Dropout(dropout),
            nn.Linear(self.dim * 4, self.dim))

    def forward(self, x, xl_memories, rel_pos):
        """
        Forward pass of the Block module.

        Args:
            x (torch.Tensor): Input tensor.
            xl_memories (torch.Tensor): XL memory for cross-attention.
            rel_pos (torch.Tensor): Tensor containing relative positions.

        Returns:
            torch.Tensor: Output tensor.
            torch.Tensor: New XL memory.
        """
        residual = x
        attn_out = self.norm(x)
        attn_out, new_xl_memories = self.attention(attn_out, relative_positions=rel_pos, xl_memory=xl_memories)
        attn_out += residual

        residual = attn_out
        ff_out = self.ff_block(attn_out)
        ff_out += residual
        return ff_out, new_xl_memories

In [ ]:
class MemorizingTransformer(nn.Module):
    """
    MemorizingTransformer module implements a transformer with memory capabilities.

    Args:
        embedding_dimension (int): Dimensionality of the input embeddings.
        vocab_size (int): Size of the vocabulary.
        max_knn_memories (int): Maximum number of KNN memories.
        heads (int): Number of attention heads.
        depth (int): Depth of the transformer.
        dropout (float): Dropout probability.
        head_dimension (int): Dimensionality of each attention head.
        topk (int): Number of top memories to retrieve.

    Attributes:
        heads (int): Number of attention heads.
        embedding_dimension (int): Dimensionality of the input embeddings.
        dropout (float): Dropout probability.
        depth (int): Depth of the transformer.
        head_dimension (int): Dimensionality of each attention head.
        max_knn_memories (int): Maximum number of KNN memories.
        topk (int): Number of top memories to retrieve.
        rel_pos (RelativePosition): Relative position embedding for XLAttention.
        rel_pos_knn (RelativePosition): Relative position embedding for KNNAttention.
        embedding_matrix (nn.Embedding): Embedding matrix.
        knn (KNN): KNN object for memory retrieval.
        layers (nn.ModuleList): List of transformer blocks.
        to_logits (nn.Sequential): Final linear layer for logits.

    Methods:
        forward: Forward pass of the MemorizingTransformer module.
    """

    def __init__(
        self,
        embedding_dimension,
        vocab_size,
        max_knn_memories=81920,
        heads=8,
        depth=10,
        dropout=0,
        head_dimension=64,
        topk=5,
    ):
        """
        Initializes the MemorizingTransformer module.

        Args:
            embedding_dimension (int): Dimensionality of the input embeddings.
            vocab_size (int): Size of the vocabulary.
            max_knn_memories (int): Maximum number of KNN memories.
            heads (int): Number of attention heads.
            depth (int): Depth of the transformer.
            dropout (float): Dropout probability.
            head_dimension (int): Dimensionality of each attention head.
            topk (int): Number of top memories to retrieve.
        """
        super().__init__()
        self.heads = heads
        self.embedding_dimension = embedding_dimension
        self.dropout = dropout
        self.depth = depth
        self.head_dimension = head_dimension
        self.max_knn_memories = max_knn_memories
        self.topk = topk

        self.rel_pos = RelativePosition(rp_scale=head_dimension ** 0.5, heads=self.heads)
        self.rel_pos_knn = RelativePosition(rp_scale=head_dimension ** 0.5, heads=self.heads)
        self.embedding_matrix = nn.Embedding(vocab_size, self.embedding_dimension)
        self.knn = KNN(head_dimension * heads, self.max_knn_memories)

        self.layers = nn.ModuleList([])
        for i in range(self.depth):
            if i == self.depth - 2:
                attention_type = KNNAttention(
                    self.embedding_dimension,
                    self.knn,
                    heads=self.heads,
                    head_dimension=self.head_dimension,
                    dropout=self.dropout
                )
            else:
                attention_type = XLAttention(
                    self.embedding_dimension,
                    heads=self.heads,
                    head_dimension=self.head_dimension,
                    dropout=self.dropout
                )
            self.layers.append(Block(self.embedding_dimension, attention_type))

        self.to_logits = nn.Sequential(
            nn.LayerNorm(self.embedding_dimension),
            nn.Linear(self.embedding_dimension, vocab_size)
        )

    def forward(
        self,
        x,
        relative_positions=None,
        xl_memories=None,
        labels=None,
    ):
        """
        Forward pass of the MemorizingTransformer module.

        Args:
            x (torch.Tensor): Input tensor.
            relative_positions (torch.Tensor): Tensor containing relative positions.
            xl_memories (torch.Tensor): XL memory for cross-attention.
            labels (torch.Tensor): Target labels.

        Returns:
            torch.Tensor: Loss tensor.
            list: List of new XL memories.
        """
        device = x.device
        batch_size, sequence_length = x.shape[0], x.shape[1]

        rel_pos = self.rel_pos(sequence_length, device=device)
        rel_pos_knn = self.rel_pos_knn(sequence_length, device=device)

        if xl_memories is not None:
            xl_memories = xl_memories
        else:
            xl_memories = (None,) * self.depth

        xl_memories_iter = iter(xl_memories)

        x = self.embedding_matrix(x)
        new_xl_memories = []

        for ind, block in enumerate(self.layers):
            if ind == self.depth - 2:
                layer_rel_pos = rel_pos_knn
            else:
                layer_rel_pos = rel_pos

            x, xl_mem = block(x, next(xl_memories_iter), layer_rel_pos)

            if xl_mem is not None:
                new_xl_memories.append(xl_mem.detach())

        logits = self.to_logits(x)
        loss = F.cross_entropy(rearrange(logits, 'b n c -> b c n'), labels)

        if len(new_xl_memories) > 0:
            return loss, new_xl_memories
        return loss


In [ ]:
import numpy as np
import torch
from torch.utils.data import DataLoader
from datasets import load_dataset

SEGMENTS = 10
SEQUENCE_LENGTH = 512
CHUNK_SIZE = (SEGMENTS * SEQUENCE_LENGTH) + 1
BATCH_SIZE = 8

# Load dataset
dataset = load_dataset("ccdv/arxiv-summarization", split='train', streaming=True)
raw_dataset = list(dataset.take(3500))

# Extract raw articles
raw_articles = [x['article'] for x in raw_dataset]
raw_articles = [x for x in raw_articles if len(x) > CHUNK_SIZE]

# Convert to numpy arrays
converted = [np.fromstring(doc, dtype=np.uint8) for doc in raw_articles]

# Clip articles to CHUNK_SIZE
def clip_article(doc, chunk_size):
    remainder = len(doc) % chunk_size
    return doc[:-remainder]

clipped = [clip_article(doc, CHUNK_SIZE) for doc in converted]



# Ensure all documents have the same shape
min_length = min(len(doc) for doc in clipped)
clipped = [doc[:min_length] for doc in clipped]

# Reshape the documents
chunked = np.array([doc.reshape(-1, CHUNK_SIZE) for doc in clipped])

# Convert to torch tensor
processed_data = torch.tensor(np.concatenate(chunked), dtype=torch.long)

# Split into train, validation, and test loaders
eighty_split = int(processed_data.shape[0] * .8)
ninety_split = int(processed_data.shape[0] * .9)

train_loader = DataLoader(processed_data[:eighty_split], batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(processed_data[eighty_split:ninety_split], batch_size=BATCH_SIZE, shuffle=True)
test_loader = DataLoader(processed_data[ninety_split:], batch_size=BATCH_SIZE, shuffle=True)

# Check the shape of processed_data
print("Processed Data Shape:", processed_data.shape)


Processed Data Shape: torch.Size([3401, 5121])


/tmp/ipykernel_34/3020860341.py:20: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead
  converted = [np.fromstring(doc, dtype=np.uint8) for doc in raw_articles]


In [ ]:
model = MemorizingTransformer(embedding_dimension = 128,
                              vocab_size = 128,
                              max_knn_memories = MAX_KNN_MEMORIES)

model.to(device) ###########

optim = torch.optim.Adam(model.parameters(), lr = LEARNING_RATE)
model.train()

for i in tqdm.tqdm(range(200), mininterval = 10., desc = 'training'):

    model.train()
    train_loss = 0.
    # Clear XL memories
    xl_memories = None
    # Clear KNN memory
    model.knn.clear()

    data = next(iter(train_loader)).to(device=device)
    seq, labels = data[:, :-1], data[:, 1:]

    t0 = time.time()
    print ("Begin document")

    # Each pass will be (BATCH_SIZE * SEGMENTS) iterations
    for seq_segment, labels_segment in zip(seq.chunk(SEGMENTS, dim = -1), labels.chunk(SEGMENTS, dim = -1)):

        loss, xl_memories = model(
            seq_segment,
            labels = labels_segment,
            xl_memories = xl_memories
        )

        train_loss += loss.item() / SEGMENTS
        (loss / SEGMENTS).backward()


    print(f'training loss: {train_loss}')
    t1 = time.time()
    print ("End document, total time:", t1 - t0)
    torch.nn.utils.clip_grad_norm_(model.parameters(), MAX_GRAD_CLIP_NORM)
    optim.step()
    optim.zero_grad()


    if not (i % VALIDATE_EVERY):
        model.eval()

        valid_data = next(iter(val_loader))
        valid_loss = 0.

        with torch.no_grad():
            xl_memories = None
            model.knn.clear()
            seq, labels = data[:, :-1], data[:, 1:]

            for seq_segment, labels_segment in zip(seq.chunk(SEGMENTS, dim = -1), labels.chunk(SEGMENTS, dim = -1)):

                loss, xl_memories = model(
                    seq_segment,
                    labels = labels_segment,
                    xl_memories = xl_memories
                )

                valid_loss += loss.item() / SEGMENTS

        print(f'valid loss: {valid_loss}')


training:   0%|          | 0/200 [00:00<?, ?it/s]

Begin document
training loss: 5.039541959762574
End document, total time: 8.374260902404785


training:   0%|          | 1/200 [00:15<50:12, 15.14s/it]

valid loss: 4.46309003829956
Begin document
training loss: 4.492447185516358
End document, total time: 7.744511842727661
Begin document


training:   2%|▏         | 3/200 [00:30<31:27,  9.58s/it]

...

training loss: 2.4208086252212526
End document, total time: 6.870612382888794
Begin document
training loss: 2.4255853891372676
End document, total time: 6.945481061935425
Begin document


training:  80%|████████  | 161/200 [20:00<04:41,  7.23s/it]

training loss: 2.397289824485779
End document, total time: 7.173508405685425
Begin document


training:  80%|████████  | 161/200 [20:04<04:51,  7.48s/it]


KeyboardInterrupt: 